In [1]:
import requests
import pandas as pd
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import matplotlib.pyplot as plt

headers = {'User-Agent': 'youremail@example.com'}  # Replace with your email

def get_cik(ticker):
    url = "https://www.sec.gov/files/company_tickers.json"
    r = requests.get(url, headers=headers)
    data = r.json()
    for entry in data.values():
        if entry['ticker'].lower() == ticker.lower():
            return str(entry['cik_str']).zfill(10)
    return None

def get_company_facts(cik):
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    r = requests.get(url, headers=headers)
    return r.json()

def extract_quarterly_facts(facts, param):
    try:
        entries = facts['facts']['us-gaap'][param]['units']['USD']
        df = pd.DataFrame(entries)
        df['end'] = pd.to_datetime(df['end'])
        df = df[df['form'].str.contains('10-Q|10-K')]
        df['quarter'] = df['end'].dt.to_period('Q')
        df = df.sort_values('quarter').drop_duplicates('quarter', keep='last')
        df = df[['quarter', 'val']].rename(columns={'val': param})
        return df
    except KeyError:
        return pd.DataFrame(columns=['quarter', param])

def extract_quarterly_revenue(facts):
    possible_tags = [
        'RevenueFromContractWithCustomerExcludingAssessedTax',
        'Revenues',
        'SalesRevenueNet',
        'TotalRevenues'
    ]
    combined_df = pd.DataFrame()
    for tag in possible_tags:
        try:
            df = extract_quarterly_facts(facts, tag)
            if combined_df.empty:
                combined_df = df
            else:
                combined_df = pd.merge(combined_df, df, on='quarter', how='outer')
        except KeyError:
            continue
    if combined_df.empty:
        return pd.DataFrame(columns=['quarter', 'Revenues'])
    for tag in possible_tags:
        if tag in combined_df.columns and not combined_df[tag].isnull().all():
            if 'Revenues' in combined_df.columns:
                combined_df['Revenues'] = combined_df['Revenues'].combine_first(combined_df[tag])
            else:
                combined_df['Revenues'] = combined_df[tag]
    return combined_df[['quarter', 'Revenues']]

def extract_quarterly_shares(facts):
    tags = [
        'CommonStockSharesOutstanding',
        'EntityCommonStockSharesOutstanding',
        'WeightedAverageNumberOfSharesOutstandingBasic',
        'WeightedAverageNumberOfDilutedSharesOutstanding'
    ]
    combined_df = pd.DataFrame()
    for tag in tags:
        try:
            entries = facts['facts']['us-gaap'][tag]['units']['shares']
            df = pd.DataFrame(entries)
            df['end'] = pd.to_datetime(df['end'])
            df = df[df['form'].str.contains('10-Q|10-K')]
            df['quarter'] = df['end'].dt.to_period('Q')
            df = df.sort_values('quarter').drop_duplicates('quarter', keep='last')
            df = df[['quarter', 'val']].rename(columns={'val': 'SharesOutstanding'})
            if combined_df.empty:
                combined_df = df
            else:
                combined_df = combined_df.combine_first(df)
        except KeyError:
            continue
    return combined_df

def get_quarterly_prices_av(ticker, api_key="8VY6A82GD5S2DKVP"):
    try:
        ts = TimeSeries(key=api_key, output_format='pandas')
        data, meta = ts.get_weekly_adjusted(symbol=ticker)
        data.index = pd.to_datetime(data.index)
        data['quarter'] = data.index.to_period('Q')
        price_df = data.groupby('quarter')['5. adjusted close'].last().reset_index()
        price_df = price_df.rename(columns={'5. adjusted close': 'YearEndPrice'})
        return price_df
    except Exception as e:
        return pd.DataFrame(columns=['quarter', 'YearEndPrice'])

def display_quarterly_fundamentals(ticker):
    output_area.clear_output()
    with output_area:
        cik = get_cik(ticker)
        if not cik:
            display(Markdown(f"**CIK not found for `{ticker}`**"))
            return

        display(Markdown(f"### CIK for `{ticker}` : `{cik}`"))

        facts = get_company_facts(cik)

        df_assets = extract_quarterly_facts(facts, 'Assets')
        df_liabilities = extract_quarterly_facts(facts, 'Liabilities')
        df_retained = extract_quarterly_facts(facts, 'RetainedEarningsAccumulatedDeficit')
        df_operating = extract_quarterly_facts(facts, 'OperatingIncomeLoss')
        df_revenue = extract_quarterly_revenue(facts)
        df_price = get_quarterly_prices_av(ticker)
        df_shares = extract_quarterly_shares(facts)

        all_dfs = [df_assets, df_liabilities, df_retained, df_operating, df_revenue, df_price, df_shares]
        for i in range(len(all_dfs)):
            if not all_dfs[i].empty and 'quarter' in all_dfs[i].columns:
                if not isinstance(all_dfs[i]['quarter'].dtype, pd.PeriodDtype):
                    all_dfs[i]['quarter'] = pd.to_datetime(all_dfs[i]['quarter'].astype(str), errors='coerce').dt.to_period('Q')

        df = all_dfs[0]
        for other_df in all_dfs[1:]:
            df = pd.merge(df, other_df, on='quarter', how='outer')

        df['WorkingCapital'] = df['Assets'] - df['Liabilities']
        df['A'] = df['WorkingCapital'] / df['Assets']
        df['B'] = df['RetainedEarningsAccumulatedDeficit'] / df['Assets']
        df['C'] = df['OperatingIncomeLoss'] / df['Assets']
        df['D'] = (df['YearEndPrice'] * df['SharesOutstanding'] / 1e9) / df['Liabilities']
        df['E'] = df['Revenues'] / df['Assets']
        df['ZScore'] = 1.2 * df['A'] + 1.4 * df['B'] + 3.3 * df['C'] + 0.6 * df['D'] + 1.0 * df['E']
        df.drop(columns=['A', 'B', 'C', 'D', 'E'], inplace=True)

        for col in ['Assets', 'Liabilities', 'RetainedEarningsAccumulatedDeficit', 'OperatingIncomeLoss', 'Revenues', 'WorkingCapital']:
            if col in df.columns:
                df[col] = df[col] / 1e9

            df = df.sort_values('quarter')
            df = df[df['quarter'] >= pd.Period('2015Q1')].reset_index(drop=True)
        import numpy as np

        df['YearEndPrice'] = (df['YearEndPrice']
            .astype(str).str.replace(r'[^\d.\-]', '', regex=True).astype(float))
        df['Return'] = np.log(df['YearEndPrice']).diff()
        eps = 1e-9  # tolerance for tiny float noise
        df = df[df['Return'].abs() > eps].reset_index(drop=True)
        
        # Set confidence level
        alpha = 0.95


        display(Markdown("### Quarterly Financials"))
        display(df.style.format({
            'Assets': '{:.2f}B',
            'Liabilities': '{:.2f}B',
            'RetainedEarningsAccumulatedDeficit': '{:.2f}B',
            'OperatingIncomeLoss': '{:.2f}B',
            'Revenues': '{:.2f}B',
            'WorkingCapital': '{:.2f}B',
            'YearEndPrice': '${:.2f}',
            'SharesOutstanding': lambda x: f'{x/1e9:.2f}B' if pd.notna(x) else '',
            'ZScore': '{:.2f}'
        }))
         # --- side-by-side layout ---
        left = widgets.Output()   # table
        right = widgets.Output()  # chart

        with right:
            zs = df[['quarter', 'ZScore']].dropna()
        if zs.empty:
            display(Markdown("**No Z-Score values available to plot.**"))
        else:
            fig, ax = plt.subplots(figsize=(9.5, 6))  # larger chart
            x = zs['quarter'].dt.to_timestamp('Q')    # Period -> datetime
            ax.plot(x, zs['ZScore'])                  # simple line chart
            ax.set_title(f'Altman Z-Score by Quarter – {ticker}')
            ax.set_xlabel('Quarter')
            ax.set_ylabel('Z-Score')
        # optional reference lines (remove if not needed)
            ax.axhline(1.8, linestyle='--')
            ax.axhline(3.0, linestyle=':')
            fig.autofmt_xdate()
            plt.tight_layout()
            display(fig)

        # put chart on the RIGHT of the table
        left.layout  = widgets.Layout(width='68%')
        right.layout = widgets.Layout(width='32%')
        display(widgets.HBox([left, right], layout=widgets.Layout(width='100%')))

        # --- Quarterly Return chart (below the Z-Score chart) ---
        ret = df[['quarter','Return']].dropna()
        if not ret.empty:
            fig2, ax2 = plt.subplots(figsize=(9.5, 3.6))  # wide, fits the right column
            x2 = ret['quarter'].dt.to_timestamp('Q')      # Period[Q] -> datetime
            ax2.plot(x2, ret['Return'])                   # line chart
            ax2.axhline(0, linestyle='--', linewidth=1)   # zero baseline
            ax2.set_title(f'Quarterly Return – {ticker}')
            ax2.set_xlabel('Quarter')
            ax2.set_ylabel('Log return')                  # or 'Simple return' if that's what you compute
            fig2.autofmt_xdate()
            plt.tight_layout()
            display(fig2)
        else:
            display(Markdown("**No Return data available to plot.**"))


# --- Widgets for Voila ---
ticker_input = widgets.Text(
    value='AAPL',
    placeholder='Enter ticker symbol (e.g., AAPL)',
    description='Ticker:',
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='Fetch Data',
    button_style='success'
)

output_area = widgets.Output()

def on_run_button_clicked(b):
    display_quarterly_fundamentals(ticker_input.value)

run_button.on_click(on_run_button_clicked)

display(Markdown("## 📊 Altman Z-Score & Financial Dashboard"))
display(ticker_input, run_button, output_area)


## 📊 Altman Z-Score & Financial Dashboard

Text(value='AAPL', description='Ticker:', placeholder='Enter ticker symbol (e.g., AAPL)', style=DescriptionSty…

Button(button_style='success', description='Fetch Data', style=ButtonStyle())

Output()